In [17]:
%%capture --no-stderr
%pip install -U langgraph langchain langchain-openai langchainhub langchain-community

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

open_ai_key = os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [2]:
## Modelo de embeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

model = ChatOpenAI(model="gpt-4o", temperature=0)
embedder = OpenAIEmbeddings()

In [4]:
! pip install pypdf

  Using cached pypdf-4.3.1-py3-none-any.whl.metadata (7.4 kB)
Using cached pypdf-4.3.1-py3-none-any.whl (295 kB)


In [5]:
from langchain_community.document_loaders import PyPDFLoader

# cargar documentos
loader = PyPDFLoader("example_data/meeting_data.pdf")
paper = loader.load()

In [ ]:
%pip install -U faiss-cpu

In [8]:
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

# text splits for the document
chunks_paper = text_splitter.split_documents(paper)

# storing in vector store
faiss_db_paper = FAISS.from_documents(chunks_paper, embedder)


In [9]:
## creating retriever

retriever_paper = faiss_db_paper.as_retriever()

In [10]:
## converting the retriever to a Tool

from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever_paper, "RAG doc", "Search for info on RAG")

In [11]:
# test it!

retriever_tool.invoke("¿Cuándo fue la reunion?")

'Reunión de planificación - 15 de agosto de 2024\nAsistentes:\n- Juan Pérez\n- María González\n- Luis Fernández\n- Ana Martínez\nAgenda:\n1. Revisión de los objetivos del trimestre pasado.\n2. Planificación de las próximas actividades.\n3. Asignación de responsabilidades.\n4. Preguntas y respuestas.\nConclusiones:\n- Se logró el 85% de los objetivos del trimestre pasado.\n- Se identificaron áreas de mejora en la comunicación interna.\n- Las actividades prioritarias para el próximo trimestre incluyen la mejora de procesos y la\ncapacitación del personal.\nLa próxima reunión está programada para el 20 de septiembre de 2024.'

In [13]:
! pip install wikipedia

  Using cached wikipedia-1.4.0-py3-none-any.whl
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)


In [14]:
# Añade otras tools para tu agente

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


# Tool para buscar en Wikipedia
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [15]:
from langchain.tools import Tool

# Documenta tus tools 
wiki_tool = Tool.from_function(
    func=wikipedia.run,
    name="wiki",
    description="useful for when you need to search certain topic on Wikipedia, aka wiki")


tools = [wiki_tool, retriever_tool] 

In [ ]:
%pip install -q httpx

In [23]:

from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation # reinstalar!

# Esta clase es útil para ejecutar las herramientas
# Toma una acción del agente y llama a esa herramienta y devuelve el resultado
tool_executor = ToolExecutor(tools)

C:\Users\Diana\AppData\Local\Temp\ipykernel_24264\2884069654.py:5: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


In [24]:

from langchain_core.messages import AIMessage
from langchain_core.tools import tool

from langgraph.prebuilt import ToolNode
tool_node = ToolNode(tools)

In [25]:
# Probar 
invocation = ToolInvocation(tool="RAG doc", tool_input="What is Retrieval Augmented Generation?")

tool_executor.invoke(invocation)

C:\Users\Diana\AppData\Local\Temp\ipykernel_24264\2915905985.py:2: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  invocation = ToolInvocation(tool="RAG doc", tool_input="What is Retrieval Augmented Generation?")


'Reunión de planificación - 15 de agosto de 2024\nAsistentes:\n- Juan Pérez\n- María González\n- Luis Fernández\n- Ana Martínez\nAgenda:\n1. Revisión de los objetivos del trimestre pasado.\n2. Planificación de las próximas actividades.\n3. Asignación de responsabilidades.\n4. Preguntas y respuestas.\nConclusiones:\n- Se logró el 85% de los objetivos del trimestre pasado.\n- Se identificaron áreas de mejora en la comunicación interna.\n- Las actividades prioritarias para el próximo trimestre incluyen la mejora de procesos y la\ncapacitación del personal.\nLa próxima reunión está programada para el 20 de septiembre de 2024.'

In [29]:
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "RAG doc",
            "args": {"query": "¿Cuándo fue la reunion?"},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

tool_node.invoke({"messages": [message_with_single_tool_call]})


{'messages': [ToolMessage(content='Reunión de planificación - 15 de agosto de 2024\nAsistentes:\n- Juan Pérez\n- María González\n- Luis Fernández\n- Ana Martínez\nAgenda:\n1. Revisión de los objetivos del trimestre pasado.\n2. Planificación de las próximas actividades.\n3. Asignación de responsabilidades.\n4. Preguntas y respuestas.\nConclusiones:\n- Se logró el 85% de los objetivos del trimestre pasado.\n- Se identificaron áreas de mejora en la comunicación interna.\n- Las actividades prioritarias para el próximo trimestre incluyen la mejora de procesos y la\ncapacitación del personal.\nLa próxima reunión está programada para el 20 de septiembre de 2024.', name='RAG doc', tool_call_id='tool_call_id')]}

In [16]:
## PROMPT TEMPLATE

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate

## set up memory
memory = ConversationBufferMemory(memory_key="chat_history", input_key='input', output_key="output")

prompt_template = """
### [INST]

Assistant is a large language model to answer questions on Retrieval Augmented Generation.


Context:
------

Assistant has access to the following tools:

{tools}

To use a tool, please use the following format:

'''
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
'''

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

'''
Thought: Do I need to use a tool? No
Final Answer: [your response here]
'''

Begin!

Previous conversation history:
{chat_history}

New input: {input}

Current Scratchpad:
{agent_scratchpad}

[/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'],
    template=prompt_template,
)

prompt = prompt.partial(
    tools=[t.name for t in tools],
    tool_names=", ".join([t.name for t in tools]),
)
print("prompt ---> \n", prompt)

prompt ---> 
 input_variables=['agent_scratchpad', 'chat_history', 'input'] partial_variables={'tools': ['wiki', 'arxiv', 'tavily', 'serpapi', 'RAG doc'], 'tool_names': 'wiki, arxiv, tavily, serpapi, RAG doc'} template="\n### [INST]\n\nAssistant is a large language model to answer questions on Retrieval Augmented Generation.\n\n\nContext:\n------\n\nAssistant has access to the following tools:\n\n{tools}\n\nTo use a tool, please use the following format:\n\n'''\nThought: Do I need to use a tool? Yes\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n'''\n\nWhen you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:\n\n'''\nThought: Do I need to use a tool? No\nFinal Answer: [your response here]\n'''\n\nBegin!\n\nPrevious conversation history:\n{chat_history}\n\nNew input: {input}\n\nCurrent Scratchpad:\n{agent_scratchpad}\n\n[/INST]\n "


In [31]:
# Crear un Agente ReAct
from langchain import hub

# Utilicemos este prompt para crear un agente
prompt = hub.pull("hwchase17/react")


In [36]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [37]:
from langchain.agents import AgentExecutor, create_react_agent

# Construye tu agente
agent = create_react_agent(model, tools, prompt)

# Crea el ejecutor de un agente pasádole los tools y el agente
agent_executor = AgentExecutor(agent=agent, 
                               tools=tools, 
                               verbose=True,
                               handle_parsing_errors=True,
                               max_iterations=3,
                               return_intermediate_steps=True,
                               early_stopping_method="generate")

In [38]:
# Probemos nuestro agente
agent_outcome = agent_executor.invoke({"input": "¿Quiénes intervienen en la reunión del 15 de agosto?","chat_history": []})
agent_outcome



> Entering new AgentExecutor chain...
Para responder a esta pregunta, necesito buscar información específica sobre la reunión del 15 de agosto. La mejor manera de encontrar esta información es buscar en una fuente confiable como Wikipedia o en documentos relevantes.

Action: wiki
Action Input: "reunión del 15 de agosto"Page: Vencer la culpa
Summary: Vencer la culpa (English: Overcoming Guilt) is a Mexican telenovela produced by Rosy Ocampo for TelevisaUnivision. The series is the fifth production of the "Vencer" franchise, and aired on Las Estrellas from 26 June 2023 to 13 October 2023. It stars Claudia Martín, Gabriela de la Garza, María Sorté and Romina Poza.

Page: Isla Entre Ríos
Summary: Entre Ríos Island (Spanish: Isla Entre Ríos) is an uninhabited island in the Paraná Delta that constitutes an exclave of Argentina in waters belonging to Paraguay. It is part of the municipality of Berón de Astrada, which covers the entire Berón de Astrada Department in the north of Corrientes P

{'input': '¿Quiénes intervienen en la reunión del 15 de agosto?',
 'chat_history': [],
 'output': 'En la reunión del 15 de agosto de 2024, intervinieron Juan Pérez, María González, Luis Fernández y Ana Martínez.',
 'intermediate_steps': [(AgentAction(tool='wiki', tool_input='reunión del 15 de agosto', log='Para responder a esta pregunta, necesito buscar información específica sobre la reunión del 15 de agosto. La mejor manera de encontrar esta información es buscar en una fuente confiable como Wikipedia o en documentos relevantes.\n\nAction: wiki\nAction Input: "reunión del 15 de agosto"'),
   'Page: Vencer la culpa\nSummary: Vencer la culpa (English: Overcoming Guilt) is a Mexican telenovela produced by Rosy Ocampo for TelevisaUnivision. The series is the fifth production of the "Vencer" franchise, and aired on Las Estrellas from 26 June 2023 to 13 October 2023. It stars Claudia Martín, Gabriela de la Garza, María Sorté and Romina Poza.\n\nPage: Isla Entre Ríos\nSummary: Entre Ríos Is

In [39]:
agent_action = agent_outcome['intermediate_steps'][0][0]
output = tool_executor.invoke(agent_action)
agent_action

AgentAction(tool='wiki', tool_input='reunión del 15 de agosto', log='Para responder a esta pregunta, necesito buscar información específica sobre la reunión del 15 de agosto. La mejor manera de encontrar esta información es buscar en una fuente confiable como Wikipedia o en documentos relevantes.\n\nAction: wiki\nAction Input: "reunión del 15 de agosto"')

In [40]:
# crea una clase AgentState para almacenar el estado del agente

from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    # el string de input del usuario
    input: str
    # Lista de mensajes previos
    chat_history: list[BaseMessage]
    # El outcome de una llamada a un agente
    agent_outcome: Union[AgentAction, AgentFinish, None]
    
    # Lista de acciones y observaciones correspondientes
    # Aquí anotamos esto con `operator.add` para indicar que las operaciones a
    # este estado deben AGREGARSE a los valores existentes (no sobrescribirlo)
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

In [41]:
# Define los nodos
from langchain_core.agents import AgentActionMessageLog

def run_agent(data):
    inputs = data.copy()
    text = inputs['input']
    agent_outcome = agent_executor.invoke({"input":text})
    return {"agent_outcome": agent_outcome}

# Define la función que ejecuta las tools
def execute_tools(data):
    # Obtén el agent outcome más reciente
    agent_output = data["agent_outcome"]
    if len(agent_output['intermediate_steps'])>=1 :
        agent_action = agent_output['intermediate_steps'][0][0]
        output = tool_executor.invoke(agent_action)
        return {"intermediate_steps": [(agent_action, str(output))]}
    else:
        return {"intermediate_steps":[]}

# Lógica para decidir los conditional edges
def should_continue(data):

    # Si el agente ha terminado, entonces devolvemos `end`
    # Esto se usará al configurar el grafo para definir el flujo
    if data["agent_outcome"]["output"] is not None:
        print(" **AgentFinish** " )
        return "end"

    # Si no, se devuelve un AgentAction
    # Aquí devolvemos la cadena `continue`
    # Esto se usará al configurar el grafo para definir el flujo
    else:
        print(" **continue** " )
        return "continue"

In [42]:
from langgraph.graph import END, StateGraph

# Define un nuevo grafo de estados
workflow = StateGraph(AgentState)

In [43]:
# Define los nodos por los que se va a ciclar
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

In [44]:

# Configura el punto de entrada como `agent`
# Esto significa que este nodo es el primero en ser llamado
workflow.set_entry_point("agent")

In [45]:
# Añade los edges condicionales
workflow.add_conditional_edges(
    "agent",
    should_continue,
  
    {
        "continue": "action",
        "end": END,
    },
)

In [46]:
# Añade un normal edge
workflow.add_edge("action", "agent")

In [47]:
# Nos devuelve un runnable de LangChain
app = workflow.compile()

In [ ]:
# from langchain_core.runnables.graph import CurveStyle, NodeColors, MermaidDrawMethod
# from IPython.display import display, HTML, Image

# display(
#     Image(
#         app.get_graph().draw_mermaid_png(
#             draw_method=MermaidDrawMethod.API,
#         )
#     )
# )

In [50]:
inputs = {"input": "¿Quiénes intervinieron en la reunión del 15 de agosto?"} 
outputs = app.invoke(inputs)



> Entering new AgentExecutor chain...
Para responder a esta pregunta, necesito buscar información específica sobre la reunión del 15 de agosto y quiénes participaron en ella. Utilizaré la herramienta de búsqueda en Wikipedia para obtener detalles sobre este evento.

Action: wiki
Action Input: "reunión del 15 de agosto participantes"Page: Acapulco Shore series 10
Summary: The tenth season of Acapulco Shore, a Mexican television programme based in Acapulco was announced on August, 2022 and premiered on September 27, 2022. Filming started in June 2022 in the coastal city of Puerto Vallarta, Jalisco, making this the second time the show is filmed in this city. Days after filming began rolling, a possible case of COVID-19 among cast members caused filming to be put on hold for a few days, before resuming. This season was announced in February 2022, via a Viacom press release. On August 28, 2022, the cast members were announced. This season marks the return of original cast member, Luis “J

In [ ]:
# inputs = {"input": "Explain what are Langchain Agents. Use appropriate metaphor to connect with real-life instances"} # ,"chat_history": []
# outputs = app.invoke(inputs)

In [51]:
outputs['agent_outcome']['output']

'En la reunión del 15 de agosto de 2024, intervinieron Juan Pérez, María González, Luis Fernández y Ana Martínez.'